In [1]:
# General imports and reading the possible trades files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from numpy.random import choice
from numpy.random import randint

from util import get_stock_n_smooth
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_csv('../data/train_possible_trades.csv')
test_df = pd.read_csv('../data/test_possible_trades.csv')

print(f"Train row count: {len(train_df)}")
print(f"Test  row count: {len(test_df)}")

Train row count: 17272
Test  row count: 9552


In [2]:
ls ../python/util.py

../python/util.py


In [3]:
def stats_table(df, days_cutoff):
    pos_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct > 0].groupby(by='trading_days').count()
    pos_df.columns = ['pos_counts']
    loss_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct < 0].groupby(by='trading_days').count()
    loss_df.columns = ['neg_counts']
    zero_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct == 0].groupby(by='trading_days').count()
    zero_df.columns = ['zero_counts']
    temp = pd.concat([loss_df, pos_df, zero_df], join="outer").reset_index()
    cols = [ 'pos_counts', 'neg_counts', 'zero_counts' ]

    stats_cnt = np.zeros((3, 4))
    
    totals = np.zeros((4,))
    total_lt_xd = 0
    print('         \t pos     neg    zero    total')
    print('         \t======  ======  =====   ======')
    print(f'< {days_cutoff} days\t', end='')
    for i, num in enumerate(temp[cols].loc[temp.trading_days < days_cutoff].sum()):
        print(num, end='\t')
        total_lt_xd += num
        totals[i] += num
    print(total_lt_xd)
    totals[3] += total_lt_xd

    total_ge_xd = 0
    print(f'>= {days_cutoff} days\t', end='')
    for i, num in enumerate(temp[cols].loc[temp.trading_days >= days_cutoff].sum()):
        print(num, end='\t')
        total_ge_xd += num
        totals[i] += num
    print(total_ge_xd)
    totals[3] += total_ge_xd

    print('         \t------\t------\t-----\t------')
    print('         \t', end='')
    for num in totals:
        print(num, end='\t')

    print('')
    print('')


    totals_pct = np.zeros((4,))
    total_lt_xd = 0
    print('         \t pos     neg    zero    total')
    print('         \t======  ======  =====   ======')
    print(f'< {days_cutoff} days\t', end='')
    for i, num in enumerate(temp[cols].loc[temp.trading_days < days_cutoff].sum()):
        num_pct = (num / totals[3])* 100
        print(f'{num_pct:.2f}', end='\t')
        total_lt_xd += num_pct
        totals_pct[i] += num_pct
    print(f'{total_lt_xd:.2f}')
    totals_pct[3] += total_lt_xd

    total_ge_xd = 0
    print(f'>= {days_cutoff} days\t', end='')
    for i, num in enumerate(temp[cols].loc[temp.trading_days >= days_cutoff].sum()):
        num_pct = (num / totals[3])* 100    
        print(f'{num_pct:.2f}', end='\t')
        total_ge_xd += num_pct
        totals_pct[i] += num_pct
    print(f'{total_ge_xd:.2f}')
    totals_pct[3] += total_ge_xd

    print('         \t------\t------\t-----\t------')
    print('         \t', end='')
    for num in totals_pct:
        print(f'{num:.2f}', end='\t')

In [133]:
def stats_table(df, days_cutoff):
    pos_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct > 0].groupby(by='trading_days').count()
    pos_df.columns = ['pos_counts']
    loss_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct < 0].groupby(by='trading_days').count()
    loss_df.columns = ['neg_counts']
    zero_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct == 0].groupby(by='trading_days').count()
    zero_df.columns = ['zero_counts']
    temp = pd.concat([loss_df, pos_df, zero_df], join="outer").reset_index()
    cols = [ 'pos_counts', 'neg_counts', 'zero_counts' ]

    stats_cnt = np.zeros((3, 4))
    totals_col = np.zeros((4,))
    total_lt_xd = 0
    for i, num in enumerate(temp[cols].loc[temp.trading_days < days_cutoff].sum()):
        stats_cnt[0,i] = num
        stats_cnt[2,i] += num
    stats_cnt[0,3] = np.sum(stats_cnt[0,0:3])

    
    for i, num in enumerate(temp[cols].loc[temp.trading_days >= days_cutoff].sum()):
        stats_cnt[1,i] = num
        stats_cnt[2,i] += num
    stats_cnt[1,3] = np.sum(stats_cnt[1,0:3])  
    stats_cnt[2,3] = stats_cnt[0,3] + stats_cnt[1,3]
    
    stats_pct = (stats_cnt.copy() / stats_cnt[2,3]) * 100
    
    print('         \t pos     neg    zero    total')
    print('         \t======  ======  =====   ======')
    print(f'< {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_cnt[0, i]:.0f}', end='\t')
        
    print(f'\n>= {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_cnt[1, i]:.0f}', end='\t') 
    
    print('\n         \t------\t------\t-----\t------')
    print('         \t', end='')
    for i in range(4):
        print(f'{stats_cnt[2, i]:.0f}', end='\t')

    print('\n\n\n')
    print('         \t pos     neg    zero    total')
    print('         \t======  ======  =====   ======')
    print(f'< {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_pct[0, i]:.2f}', end='\t')
        
    print(f'\n>= {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_pct[1, i]:.2f}', end='\t') 
    
    print('\n         \t------\t------\t-----\t------')
    print('         \t', end='')
    for i in range(4):
        print(f'{stats_pct[2, i]:.2f}', end='\t')
    return stats_cnt, stats_pct

In [4]:
stats_table(test_df, 60)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 60 days	5033.0	1063.0	30.0	6126.0
>= 60 days	1350.0	2074.0	2.0	3426.0
         	------	------	-----	------
         	6383.0	3137.0	32.0	9552.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 60 days	52.69	11.13	0.31	64.13
>= 60 days	14.13	21.71	0.02	35.87
         	------	------	-----	------
         	66.82	32.84	0.34	100.00	

In [5]:
stats_table(test_df, 55)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	4875.0	948.0	26.0	5849.0
>= 55 days	1508.0	2189.0	6.0	3703.0
         	------	------	-----	------
         	6383.0	3137.0	32.0	9552.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	51.04	9.92	0.27	61.23
>= 55 days	15.79	22.92	0.06	38.77
         	------	------	-----	------
         	66.82	32.84	0.34	100.00	

In [6]:
stats_table(train_df, 60)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 60 days	17272.0	0.0	0.0	17272.0
>= 60 days	0.0	0.0	0.0	0.0
         	------	------	-----	------
         	17272.0	0.0	0.0	17272.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 60 days	100.00	0.00	0.00	100.00
>= 60 days	0.00	0.00	0.00	0.00
         	------	------	-----	------
         	100.00	0.00	0.00	100.00	

In [7]:
stats_table(train_df, 55)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	17272.0	0.0	0.0	17272.0
>= 55 days	0.0	0.0	0.0	0.0
         	------	------	-----	------
         	17272.0	0.0	0.0	17272.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	100.00	0.00	0.00	100.00
>= 55 days	0.00	0.00	0.00	0.00
         	------	------	-----	------
         	100.00	0.00	0.00	100.00	

In [8]:
def random_stock(tickers):
    while True:
        ticker = choice(tickers)
        success, hist = get_stock_n_smooth(ticker, '10y')
        if success == False:
            continue
        
        n_len = len(hist)
        if n_len < 5 * 252:
            continue
        
        train_len = int(n_len * 0.7)
        test_len  = n_len - train_len
        
        return ticker, hist.reset_index(), train_len, test_len

In [9]:
tickers = list(set(train_df.ticker) | set(test_df.ticker))
len(tickers)

2185

In [10]:
ticker, hist, train_len, test_len = random_stock(tickers)
hist = hist
ticker, len(hist), train_len, test_len

('CBZ', 2516, 1761, 755)

In [11]:
tdl   = test_df.groupby(by='trading_days').count()['ticker'].index.tolist()
p_tdl = test_df.groupby(by='trading_days').count()['ticker']
som = sum(p_tdl)
p_tdl = np.array(p_tdl / som)

In [12]:
len(tdl), len(p_tdl)

(406, 406)

In [13]:
def random_trading_days(test_len, tdl, p_tdl):
    while True:
        sday = randint(0, test_len)
        tday = choice(a=tdl, size=1, p=p_tdl)[0]
        
        if sday + tday >= test_len:
            continue
            
        return sday, tday

In [14]:
ticker, hist, train_len, test_len = random_stock(tickers)
sday, tday = random_trading_days(test_len, tdl, p_tdl)

buy_date   = str(hist['Date'].iloc[train_len+sday])[:10]
buy_close  = float(hist['Close'].iloc[train_len+sday])

sell_date  = str(hist['Date'].iloc[train_len+sday+tday])[:10]
sell_close = float(hist['Close'].iloc[train_len+sday+tday])

gain_pct   = ((sell_close - buy_close) / buy_close)
daily_ret  = (1 + gain_pct) ** (1/tday) - 1.0
gain_pct  *= 100
daily_ret *= 100

dict_df = { 'buy_close': [buy_close], 'buy_date': [buy_date], 'daily_return': [daily_ret], 
            'gain_pct' : [gain_pct],  'sell_close': [sell_close], 'sell_date': [sell_date],
            'ticker': [ticker], 'trading_days' : [tday]
          }
df = pd.DataFrame().from_dict(dict_df)
df

,buy_close,buy_date,daily_return,gain_pct,sell_close,sell_date,ticker,trading_days
0,7.82,2019-03-12,-0.129711,-2.941176,7.59,2019-04-12,CECE,23


In [15]:
def simulate_trades(nr_trades):
    sim_df = pd.DataFrame()
    for t in tqdm(range(nr_trades), 'Simulations:'):
        
        ticker, hist, train_len, test_len = random_stock(tickers)
        sday, tday = random_trading_days(test_len, tdl, p_tdl)

        buy_date   = str(hist['Date'].iloc[train_len+sday])[:10]
        buy_close  = float(hist['Close'].iloc[train_len+sday])

        sell_date  = str(hist['Date'].iloc[train_len+sday+tday])[:10]
        sell_close = float(hist['Close'].iloc[train_len+sday+tday])

        gain_pct   = ((sell_close - buy_close) / buy_close)
        daily_ret  = (1 + gain_pct) ** (1/tday) - 1.0
        gain_pct  *= 100
        daily_ret *= 100

        dict_df = { 'buy_close': [buy_close], 'buy_date': [buy_date], 'daily_return': [daily_ret], 
                    'gain_pct' : [gain_pct],  'sell_close': [sell_close], 'sell_date': [sell_date],
                    'ticker': [ticker], 'trading_days' : [tday]
                  }
        df = pd.DataFrame().from_dict(dict_df)
        sim_df = pd.concat([sim_df, df])
    
    return sim_df

In [16]:
%time sim_df = simulate_trades(len(test_df))
sim_df

Simulations::   1%|          | 85/9552 [00:31<1:00:09,  2.62it/s]

- PEIX: No data found, symbol may be delisted


Simulations::   2%|▏         | 178/9552 [01:06<1:04:36,  2.42it/s]

- PEIX: No data found, symbol may be delisted


Simulations::   8%|▊         | 736/9552 [08:33<2:37:41,  1.07s/it]

...

Simulations::  99%|█████████▉| 9437/9552 [2:37:56<01:47,  1.07it/s]

- AAXN: No data found, symbol may be delisted


Simulations:: 100%|██████████| 9552/9552 [2:39:54<00:00,  1.00s/it]

CPU times: user 11min 38s, sys: 20 s, total: 11min 58s
Wall time: 2h 39min 54s


,buy_close,buy_date,daily_return,gain_pct,sell_close,sell_date,ticker,trading_days
0,20.6500,2019-08-29,1.910885,23.147700,25.430,2019-09-16,TGI,11
0,15.3901,2018-09-11,-0.127514,-4.854419,14.643,2018-11-05,MITT,39
0,114.7400,2019-12-26,0.009390,0.697229,115.540,2020-04-14,PFPT,74
0,140.1700,2019-12-05,0.151725,28.422630,180.010,2020-08-03,IPGP,165
0,14.9900,2019-10-11,0.079291,5.203469,15.770,2020-01-14,ZEUS,64
...,...,...,...,...,...,...,...,...
0,3.3600,2018-12-12,-0.429333,-6.250000,3.150,2019-01-04,ATLC,15
0,39.2200,2019-06-07,0.306004,24.987251,49.020,2019-09-20,HRI,73
0,3.3500,2019-08-14,-0.520067,-14.925373,2.850,2019-09-27,ARAY,31
0,23.5000,2018-08-28,0.048956,0.638298,23.650,2018-09-17,CBZ,13


In [17]:
stats_table(sim_df, 55)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	3412.0	2745.0	45.0	6202.0
>= 55 days	1819.0	1526.0	5.0	3350.0
         	------	------	-----	------
         	5231.0	4271.0	50.0	9552.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	35.72	28.74	0.47	64.93
>= 55 days	19.04	15.98	0.05	35.07
         	------	------	-----	------
         	54.76	44.71	0.52	100.00	

In [18]:
stats_table(test_df, 55)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	4875.0	948.0	26.0	5849.0
>= 55 days	1508.0	2189.0	6.0	3703.0
         	------	------	-----	------
         	6383.0	3137.0	32.0	9552.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	51.04	9.92	0.27	61.23
>= 55 days	15.79	22.92	0.06	38.77
         	------	------	-----	------
         	66.82	32.84	0.34	100.00	

In [19]:
len(test_df)

9552

In [20]:
len(test_df.ticker.unique())

1990

In [21]:
len(sim_df.ticker.unique())

2146

In [22]:
len(tickers)

2185

In [23]:
%time sim_df = simulate_trades(len(test_df))
stats_table(sim_df, 55)

Simulations::   2%|▏         | 192/9552 [03:22<2:39:35,  1.02s/it]

- BEAT: No data found, symbol may be delisted


Simulations::   3%|▎         | 332/9552 [05:47<2:15:19,  1.14it/s]

- EIGI: No data found, symbol may be delisted


Simulations::   4%|▍         | 407/9552 [07:03<2:42:34,  1.07s/it]

...

- BEAT: No data found, symbol may be delisted


Simulations::  99%|█████████▉| 9502/9552 [2:41:41<00:50,  1.01s/it]

- PEIX: No data found, symbol may be delisted


Simulations:: 100%|██████████| 9552/9552 [2:42:33<00:00,  1.02s/it]

CPU times: user 11min 38s, sys: 20 s, total: 11min 58s
Wall time: 2h 42min 33s
         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	3445.0	2782.0	42.0	6269.0
>= 55 days	1774.0	1502.0	7.0	3283.0
         	------	------	-----	------
         	5219.0	4284.0	49.0	9552.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	36.07	29.12	0.44	65.63
>= 55 days	18.57	15.72	0.07	34.37
         	------	------	-----	------
         	54.64	44.85	0.51	100.00	

In [24]:
%time sim_df = simulate_trades(len(test_df))
stats_table(sim_df, 55)

Simulations::   1%|          | 114/9552 [01:53<2:07:41,  1.23it/s]

- SALT: No data found, symbol may be delisted


Simulations::   3%|▎         | 281/9552 [04:43<2:50:22,  1.10s/it]

- PEIX: No data found, symbol may be delisted


Simulations::   6%|▌         | 591/9552 [10:04<2:32:46,  1.02s/it]

...

- SALT: No data found, symbol may be delisted


Simulations::  99%|█████████▉| 9482/9552 [2:43:21<01:12,  1.04s/it]

- BEAT: No data found, symbol may be delisted


Simulations:: 100%|██████████| 9552/9552 [2:44:33<00:00,  1.03s/it]

CPU times: user 11min 38s, sys: 19.9 s, total: 11min 58s
Wall time: 2h 44min 33s
         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	3381.0	2777.0	46.0	6204.0
>= 55 days	1818.0	1527.0	3.0	3348.0
         	------	------	-----	------
         	5199.0	4304.0	49.0	9552.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	35.40	29.07	0.48	64.95
>= 55 days	19.03	15.99	0.03	35.05
         	------	------	-----	------
         	54.43	45.06	0.51	100.00	

In [25]:
pwd

'/Users/frkornet/CDA/Project/stockie/notebook'

In [28]:
fnm = '/Users/frkornet/CDA/Project/stockie/data/negtrades/test_possible_trades_05.csv'
negtrades_test_possible_trades = pd.read_csv(fnm)
negtrades_test_possible_trades

,buy_close,buy_date,daily_return,gain_pct,sell_close,sell_date,ticker,trading_days
0,65.75,2018-03-22,0.04,6.66,70.13,2018-11-28,A,174
1,62.10,2018-12-21,0.75,11.82,69.44,2019-01-15,A,15
2,66.66,2019-05-15,0.30,7.41,71.60,2019-06-19,A,24
3,68.74,2019-07-17,0.16,5.64,72.62,2019-09-05,A,35
4,81.90,2020-01-31,-0.09,-4.37,78.32,2020-04-16,A,52
...,...,...,...,...,...,...,...,...
10902,2.77,2018-12-04,0.32,6.50,2.95,2019-01-04,ZYXI,20
10903,7.70,2019-07-15,1.06,27.53,9.82,2019-08-15,ZYXI,23
10904,9.76,2019-08-22,-0.22,-3.69,9.40,2019-09-17,ZYXI,17
10905,9.01,2019-10-30,0.26,21.86,10.98,2020-02-20,ZYXI,76


In [30]:
stats_table(negtrades_test_possible_trades, 55)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	5845.0	1194.0	29.0	7068.0
>= 55 days	1515.0	2316.0	8.0	3839.0
         	------	------	-----	------
         	7360.0	3510.0	37.0	10907.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	53.59	10.95	0.27	64.80
>= 55 days	13.89	21.23	0.07	35.20
         	------	------	-----	------
         	67.48	32.18	0.34	100.00	

In [134]:
new_test_df = pd.read_csv('/Users/frkornet/CDA/Project/stockie/data/test_possible_trades.csv')
new_test_df

,buy_close,buy_date,daily_return,gain_pct,sell_close,sell_date,ticker,trading_days
0,39.78,2018-03-23,0.27,33.11,52.95,2018-08-27,AAPL,108
1,52.56,2018-10-26,-0.09,-8.83,47.92,2019-04-04,AAPL,108
2,44.93,2019-05-22,0.31,46.38,65.77,2019-11-15,AAPL,124
3,67.85,2020-02-27,0.18,8.80,73.82,2020-05-05,AAPL,47
4,37.83,2018-10-24,0.00,0.79,38.13,2019-07-05,ABCB,173
...,...,...,...,...,...,...,...,...
1649,7.74,2019-08-16,-0.18,-10.21,6.95,2019-11-08,ZIXI,59
1650,3.92,2020-03-17,1.53,72.70,6.77,2020-05-07,ZIXI,36
1651,24.25,2018-05-22,0.28,18.35,28.70,2018-08-16,ZUMZ,60
1652,27.00,2018-09-25,-0.23,-15.22,22.89,2019-01-10,ZUMZ,73


In [137]:
stats_table(new_test_df, 55)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	786	135	2	923	
>= 55 days	361	370	0	731	
         	------	------	-----	------
         	1147	505	2	1654	



         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	47.52	8.16	0.12	55.80	
>= 55 days	21.83	22.37	0.00	44.20	
         	------	------	-----	------
         	69.35	30.53	0.12	100.00	

(array([[ 786.,  135.,    2.,  923.],
        [ 361.,  370.,    0.,  731.],
        [1147.,  505.,    2., 1654.]]),
 array([[ 47.52116082,   8.16203144,   0.12091898,  55.80411125],
        [ 21.82587666,  22.37001209,   0.        ,  44.19588875],
        [ 69.34703748,  30.53204353,   0.12091898, 100.        ]]))

In [35]:
len(test_df)

9552

In [140]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

stuff = ['MACD', 'PctDiff', 'StdDev', 'RSI', 'WPR', 'MFI', 'BBP']
count_f = 0
combs = []
for p in powerset(stuff):
    if len(p) > 2:
        count_f += 1
        combs.append(p)
        print(p, len(p))
print(f"Combinations with at least 3 or more lements: {count_f}")

('MACD', 'PctDiff', 'StdDev') 3
('MACD', 'PctDiff', 'RSI') 3
('MACD', 'PctDiff', 'WPR') 3
('MACD', 'PctDiff', 'MFI') 3
('MACD', 'PctDiff', 'BBP') 3
('MACD', 'StdDev', 'RSI') 3
('MACD', 'StdDev', 'WPR') 3
('MACD', 'StdDev', 'MFI') 3
('MACD', 'StdDev', 'BBP') 3
('MACD', 'RSI', 'WPR') 3
('MACD', 'RSI', 'MFI') 3
('MACD', 'RSI', 'BBP') 3
('MACD', 'WPR', 'MFI') 3
('MACD', 'WPR', 'BBP') 3
('MACD', 'MFI', 'BBP') 3
('PctDiff', 'StdDev', 'RSI') 3
('PctDiff', 'StdDev', 'WPR') 3
('PctDiff', 'StdDev', 'MFI') 3
('PctDiff', 'StdDev', 'BBP') 3
('PctDiff', 'RSI', 'WPR') 3
('PctDiff', 'RSI', 'MFI') 3
('PctDiff', 'RSI', 'BBP') 3
('PctDiff', 'WPR', 'MFI') 3
('PctDiff', 'WPR', 'BBP') 3
('PctDiff', 'MFI', 'BBP') 3
('StdDev', 'RSI', 'WPR') 3
('StdDev', 'RSI', 'MFI') 3
('StdDev', 'RSI', 'BBP') 3
('StdDev', 'WPR', 'MFI') 3
('StdDev', 'WPR', 'BBP') 3
('StdDev', 'MFI', 'BBP') 3
('RSI', 'WPR', 'MFI') 3
('RSI', 'WPR', 'BBP') 3
('RSI', 'MFI', 'BBP') 3
('WPR', 'MFI', 'BBP') 3
('MACD', 'PctDiff', 'StdDev', 'RSI') 4
(

In [141]:
combs

[('MACD', 'PctDiff', 'StdDev'),
 ('MACD', 'PctDiff', 'RSI'),
 ('MACD', 'PctDiff', 'WPR'),
 ('MACD', 'PctDiff', 'MFI'),
 ('MACD', 'PctDiff', 'BBP'),
 ('MACD', 'StdDev', 'RSI'),
 ('MACD', 'StdDev', 'WPR'),
 ('MACD', 'StdDev', 'MFI'),
 ('MACD', 'StdDev', 'BBP'),
 ('MACD', 'RSI', 'WPR'),
 ('MACD', 'RSI', 'MFI'),
 ('MACD', 'RSI', 'BBP'),
 ('MACD', 'WPR', 'MFI'),
 ('MACD', 'WPR', 'BBP'),
 ('MACD', 'MFI', 'BBP'),
 ('PctDiff', 'StdDev', 'RSI'),
 ('PctDiff', 'StdDev', 'WPR'),
 ('PctDiff', 'StdDev', 'MFI'),
 ('PctDiff', 'StdDev', 'BBP'),
 ('PctDiff', 'RSI', 'WPR'),
 ('PctDiff', 'RSI', 'MFI'),
 ('PctDiff', 'RSI', 'BBP'),
 ('PctDiff', 'WPR', 'MFI'),
 ('PctDiff', 'WPR', 'BBP'),
 ('PctDiff', 'MFI', 'BBP'),
 ('StdDev', 'RSI', 'WPR'),
 ('StdDev', 'RSI', 'MFI'),
 ('StdDev', 'RSI', 'BBP'),
 ('StdDev', 'WPR', 'MFI'),
 ('StdDev', 'WPR', 'BBP'),
 ('StdDev', 'MFI', 'BBP'),
 ('RSI', 'WPR', 'MFI'),
 ('RSI', 'WPR', 'BBP'),
 ('RSI', 'MFI', 'BBP'),
 ('WPR', 'MFI', 'BBP'),
 ('MACD', 'PctDiff', 'StdDev', 'RSI'),
 

False

True

In [130]:
def stats_table2(df, days_cutoff):
    pos_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct > 0].groupby(by='trading_days').count()
    pos_df.columns = ['pos_counts']
    loss_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct < 0].groupby(by='trading_days').count()
    loss_df.columns = ['neg_counts']
    zero_df = df[['gain_pct', 'trading_days']].loc[df.gain_pct == 0].groupby(by='trading_days').count()
    zero_df.columns = ['zero_counts']
    temp = pd.concat([loss_df, pos_df, zero_df], join="outer").reset_index()
    cols = [ 'pos_counts', 'neg_counts', 'zero_counts' ]

    stats_cnt = np.zeros((3, 4))
    totals_col = np.zeros((4,))
    total_lt_xd = 0
    for i, num in enumerate(temp[cols].loc[temp.trading_days < days_cutoff].sum()):
        stats_cnt[0,i] = num
        stats_cnt[2,i] += num
    stats_cnt[0,3] = np.sum(stats_cnt[0,0:3])

    
    for i, num in enumerate(temp[cols].loc[temp.trading_days >= days_cutoff].sum()):
        stats_cnt[1,i] = num
        stats_cnt[2,i] += num
    stats_cnt[1,3] = np.sum(stats_cnt[1,0:3])  
    stats_cnt[2,3] = stats_cnt[0,3] + stats_cnt[1,3]
    
    stats_pct = (stats_cnt.copy() / stats_cnt[2,3]) * 100
    
    print('         \t pos     neg    zero    total')
    print('         \t======  ======  =====   ======')
    print(f'< {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_cnt[0, i]:.0f}', end='\t')
        
    print(f'\n>= {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_cnt[1, i]:.0f}', end='\t') 
    
    print('\n         \t------\t------\t-----\t------')
    print('         \t', end='')
    for i in range(4):
        print(f'{stats_cnt[2, i]:.0f}', end='\t')

    print('\n\n\n')
    print('         \t pos     neg    zero    total')
    print('         \t======  ======  =====   ======')
    print(f'< {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_pct[0, i]:.2f}', end='\t')
        
    print(f'\n>= {days_cutoff} days\t', end='')
    for i in range(4):
        print(f'{stats_pct[1, i]:.2f}', end='\t') 
    
    print('\n         \t------\t------\t-----\t------')
    print('         \t', end='')
    for i in range(4):
        print(f'{stats_pct[2, i]:.2f}', end='\t')
    return stats_cnt, stats_pct

In [131]:
stats_cnt, stats_pct = stats_table2(new_test_df, 55)
print('\n\n\n')
for i in range(3):
    print('\t\t', end='')
    for j in range(4):
        print(f'{stats_cnt[i,j]:.0f}\t', end='')
    print('')

print('')

for i in range(3):
    print('\t\t', end='')
    for j in range(4):
        print(f'{stats_pct[i,j]:.2f}\t', end='')
    print('')

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	1033	201	1	1235	
>= 55 days	435	486	3	924	
         	------	------	-----	------
         	1468	687	4	2159	



         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	47.85	9.31	0.05	57.20	
>= 55 days	20.15	22.51	0.14	42.80	
         	------	------	-----	------
         	67.99	31.82	0.19	100.00	



		1033	201	1	1235	
		435	486	3	924	
		1468	687	4	2159	

		47.85	9.31	0.05	57.20	
		20.15	22.51	0.14	42.80	
		67.99	31.82	0.19	100.00	


In [132]:
stats_table(new_test_df, 55)

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	1033.0	201.0	1.0	1235.0
>= 55 days	435.0	486.0	3.0	924.0
         	------	------	-----	------
         	1468.0	687.0	4.0	2159.0	

         	 pos     neg    zero    total
         	======  ======  =====   ======
< 55 days	47.85	9.31	0.05	57.20
>= 55 days	20.15	22.51	0.14	42.80
         	------	------	-----	------
         	67.99	31.82	0.19	100.00	

In [ ]:
stats_cnt = np.zeros((3,4))
for i, num in enumerate(temp[cols].loc[temp.trading_days < days_cutoff].sum()):
        print(num, end='\t')
        total_lt_xd += num
        totals[i] += num
    print(total_lt_xd)
    totals[3] += total_lt_xd